In [1]:
import pandas as pd
import numpy as np
import tgt
import re
import os
import pathlib

In [2]:
motion_data_path = 'mp_data'
def listdir(folder):
    for x in os.listdir(folder):
        if x != '.DS_Store':
            yield x

In [3]:
# concatenate the motion data from all sessions
sessions_mp_dataframe = pd.DataFrame()
for i, file in enumerate(sorted(os.listdir("mp_data"))):
    if file.endswith(".csv"):
        csv_path = os.path.join("mp_data", file)
        #print(csv_path)
        
        df = pd.read_csv(csv_path) 
        
        df.set_axis(['Wrist'], axis="columns", inplace=True)
        df = df['Wrist'].str.split("\t", expand = True)
        df.set_axis(["Frame", "Wrist_X", "Wrist_Y"], axis="columns", inplace=True)
        df_a = df[((df['Wrist_X'].astype(float)) * 720) <= 400] # to capture the person on the left
        df_b = df[((df['Wrist_X'].astype(float)) * 720) >= 320] # to capture the person on the right
        
        # Frame to timestamp conversion
        df_a = df_a.astype(float)
        df_b = df_b.astype(float)

        df_a["Frame"] *= float(0.04)
        df_b["Frame"] *= float(0.04)
        
        # put i_A column to df_a and i_B column to df_b merge them and put session number i
        df_a['Participant'] = str(str(i+1) + '_A')
        df_b['Participant'] = str(str(i+1) + '_B')
        
        df_both = pd.concat([df_a, df_b])
        
        df_both['Session'] = str(i+1)
        
        sessions_mp_dataframe = sessions_mp_dataframe.append(df_both)
        

/var/folders/40/svl23ydd4b52mf_1h3gxqk3m0000gn/T/ipykernel_36732/2853365032.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['Wrist'], axis="columns", inplace=True)
/var/folders/40/svl23ydd4b52mf_1h3gxqk3m0000gn/T/ipykernel_36732/2853365032.py:12: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(["Frame", "Wrist_X", "Wrist_Y"], axis="columns", inplace=True)
/var/folders/40/svl23ydd4b52mf_1h3gxqk3m0000gn/T/ipykernel_36732/2853365032.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sessions_mp_dataframe = sessions_mp_dataframe.append(df_both)
/var/folders/40/svl23ydd4b52mf_1h3gxqk3m0000gn/T/ipykernel_36732/2853365032.py:10: FutureWarning: DataFrame.

In [4]:
sessions_mp_dataframe.to_csv('all_sessions_mp_data_2.csv')

In [4]:
# Read the TextGrid
def read_tgt(session):

    filepath = 'transcriptions_annotations/' + session

    tg_path = filepath + '/' + session + '.TextGrid'
    tg = tgt.read_textgrid(tg_path) # my text grid files
    print(tg_path)
    return tg 

In [5]:
def load_session_annotation(doc_no):
    tg = read_tgt(doc_no)
    return tg    

In [6]:
def get_session_start_end(tg):
    
    part = tg.get_tier_by_name('Part')
    part_intervals = part.intervals
    part.end_time

    session_start = part.intervals[0].start_time 
    session_end = part.intervals[0].end_time
    
    return { 'session_start': session_start, 'session_end': session_end } 

In [7]:
def load_participant_annotation(sess, participant):
    # works well 
    session_id = int(sess[-1:])
    participant_id = str(session_id) +'_' + participant
    participant_utts = str(participant) + '-utts' 
    
    session_annotation = load_session_annotation(sess)
    session_start_end = get_session_start_end(session_annotation)
    session_start = float(session_start_end['session_start'])
    session_end = float(session_start_end['session_end'])
    
    participant_annotation = session_annotation.get_tier_by_name(participant_utts)
    participant_annotation.delete_annotations_with_text(pattern='laughter', n=0, regex=True) # filter laughter
    participant_partials = participant_annotation.get_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
    
    # nested dictionary or nested list
    
    partials = {} 
    fluents = {}
    
    d = {
        'session_id': session_id,
        'session_start': session_start,
        'session_end': session_end,
        'participant_id': participant_id,
        'partials': partials,
        'fluents': fluents
    }
    
    for i, interval in enumerate(participant_partials):
        # if interval.start_time in np.arange(session_start, session_end):
        start_time = interval.start_time
        # if interval.end_time in np.arange(session_start, session_end): # does not work perhaps for i, in ranger
        end_time = interval.end_time 
        partial_no = i
        disfluency = 1 
        speaker = 1
        partials[i] = {'speaker':speaker, 'partial_no': partial_no, 'disfluency':disfluency, 'start_time': start_time, 'end_time': end_time}
        
    participant_annotation.delete_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
    participant_annotation.delete_annotations_with_text(pattern='<p=', n=0, regex=True)
    participant_annotation.delete_annotations_with_text(pattern='breathing', n=0, regex=True)
    participant_fluent_intervals = participant_annotation
    
    for j, interval_flu in enumerate(participant_fluent_intervals):
        start_time = interval_flu.start_time
        end_time = interval_flu.end_time 
        fluent_no = j
        disfluency = 0 
        speaker = 1
        fluents[j] = {'speaker':speaker, 'fluent_no': fluent_no, 'disfluency':disfluency, 'start_time': start_time, 'end_time': end_time}
        
    return d

In [8]:
all_sessions = {1: 'r1', 2: 'r2', 3:'r3', 4:'r4', 5:'r5', 6:'r6', 7:'r7', 8:'r8', 9:'r9'}
participants = {'A', 'B'}

dict_all = []
for sessions in all_sessions:
    for participant in participants:
        dict_all.append(load_participant_annotation(all_sessions[int(sessions)], participant))

dict_all # works with append. So should work with yield, but it is pretty fast so why yield?

transcriptions_annotations/r1/r1.TextGrid
transcriptions_annotations/r1/r1.TextGrid
transcriptions_annotations/r2/r2.TextGrid
transcriptions_annotations/r2/r2.TextGrid
transcriptions_annotations/r3/r3.TextGrid
transcriptions_annotations/r3/r3.TextGrid
transcriptions_annotations/r4/r4.TextGrid
transcriptions_annotations/r4/r4.TextGrid
transcriptions_annotations/r5/r5.TextGrid
transcriptions_annotations/r5/r5.TextGrid
transcriptions_annotations/r6/r6.TextGrid
transcriptions_annotations/r6/r6.TextGrid
transcriptions_annotations/r7/r7.TextGrid
transcriptions_annotations/r7/r7.TextGrid
transcriptions_annotations/r8/r8.TextGrid
transcriptions_annotations/r8/r8.TextGrid
transcriptions_annotations/r9/r9.TextGrid
transcriptions_annotations/r9/r9.TextGrid


[{'session_id': 1,
  'session_start': 547.168380434,
  'session_end': 1321.59253469,
  'participant_id': '1_A',
  'partials': {0: {'speaker': 1,
    'partial_no': 0,
    'disfluency': 1,
    'start_time': 605.446997075,
    'end_time': 607.403301579},
   1: {'speaker': 1,
    'partial_no': 1,
    'disfluency': 1,
    'start_time': 640.866221459,
    'end_time': 643.835303074},
   2: {'speaker': 1,
    'partial_no': 2,
    'disfluency': 1,
    'start_time': 659.644876156,
    'end_time': 662.754119798},
   3: {'speaker': 1,
    'partial_no': 3,
    'disfluency': 1,
    'start_time': 669.377786575,
    'end_time': 681.827085792},
   4: {'speaker': 1,
    'partial_no': 4,
    'disfluency': 1,
    'start_time': 691.41017797,
    'end_time': 693.80794942},
   5: {'speaker': 1,
    'partial_no': 5,
    'disfluency': 1,
    'start_time': 722.804262555,
    'end_time': 723.47079108},
   6: {'speaker': 1,
    'partial_no': 6,
    'disfluency': 1,
    'start_time': 750.476174853,
    'end_time':

In [ ]:
  for p in participants:
        if p['session_id'] == me['session_id'] and p['participant_id'] != me['participant_id']:
            return p

In [9]:
all_records = []

for i, participant in enumerate(dict_all):
    participant_id = dict_all[i]['participant_id']
    # print(participant) # contains all dictionary of the participant
    print(participant_id)
    mp_participant = sessions_mp_dataframe[(sessions_mp_dataframe['Participant'] == participant_id)]
    mp_times = mp_participant['Frame']
        
    partials = dict_all[i]['partials']
    
    for partial in partials:
        start = partials[partial]['start_time']
        end = partials[partial]['end_time']
        partial_no = partials[partial]['partial_no']
        for index, row in mp_participant.iterrows():
            time = row['Frame']
            hand_position_X = row['Wrist_X']
            hand_position_Y = row['Wrist_Y']   
            if start <= time and time <= end:
                disfluency = 1
                mp_participant_partial_handX = hand_position_X 
                mp_participant_partial_handY = hand_position_Y
                all_records.append([participant_id, disfluency, partial_no, mp_participant_partial_handX, mp_participant_partial_handY])
     
    fluents = dict_all[i]['fluents']
    
    for fluent in fluents:
        start2 = fluents[fluent]['start_time']
        end2 = fluents[fluent]['end_time']
        fluent_no = fluents[fluent]['fluent_no']
        for index2, row2 in mp_participant.iterrows():
            time2 = row2['Frame']
            hand_position_X2 = row2['Wrist_X']
            hand_position_Y2 = row2['Wrist_Y'] 
            if start2 <= time2 and time2 <= end2:           
                disfluency = 0
                mp_participant_fluent_handX = hand_position_X2
                mp_participant_fluent_handY = hand_position_Y2
                all_records.append([participant_id, disfluency, fluent_no, mp_participant_fluent_handX, mp_participant_fluent_handY]) 

                
all_records_dataframe = pd.DataFrame(all_records, columns = ['ParticipantID', 'Disfluency', 'EventNo', 'Wrist_X', 'Wrist_Y'])


1_A
1_B
2_A
2_B
3_A
3_B
4_A
4_B
5_A
5_B
6_A
6_B
7_A
7_B
8_A
8_B
9_A
9_B


In [10]:
all_records_dataframe.to_csv('all_records_speakers.csv')

In [35]:
all_records_dataframe

,ParticipantID,Disfluency,EventNo,Wrist_X,Wrist_Y
0,1_A,1,1,0.244067,0.370307
1,1_A,1,1,0.241562,0.360925
2,1_A,1,1,0.244293,0.355284
3,1_A,1,1,0.258994,0.380093
4,1_A,1,1,0.258011,0.370002
...,...,...,...,...,...
137139,9_B,0,218,0.801131,0.701714
137140,9_B,0,218,0.801131,0.701714
137141,9_B,0,218,0.801131,0.701714
137142,9_B,0,218,0.801131,0.701714


In [ ]:
def find_partner(participants, me):
    for p in participants:
        if p['session_id'] == me['session_id'] and p['participant_id'] != me['participant_id']:
            return p

In [12]:
### listener

all_records_listener = []

for i, participant in enumerate(dict_all):
    participant_id = dict_all[i]['participant_id']
    session_id = dict_all[i]['session_id']
    

    # print(participant) # contains all dictionary of the participant
    print(participant_id)
    mp_participant = sessions_mp_dataframe[(sessions_mp_dataframe['Session'] == session_id) and (sessions_mp_dataframe['Participant'] != participant_id)]
    print(mp_participant['Participant']) # should be the listener
    
    

1_A


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
    mp_times = mp_participant['Frame']
        
    partials = dict_all[i]['partials']
    
    for partial in partials:
        start = partials[partial]['start_time']
        end = partials[partial]['end_time']
        partial_no = partials[partial]['partial_no']
        for index, row in mp_participant.iterrows():
            time = row['Frame']
            hand_position_X = row['Wrist_X']
            hand_position_Y = row['Wrist_Y']   
            if start <= time and time <= end:
                disfluency = 1
                mp_participant_partial_handX = hand_position_X 
                mp_participant_partial_handY = hand_position_Y
                all_records.append([participant_id, disfluency, partial_no, mp_participant_partial_handX, mp_participant_partial_handY])
     
    fluents = dict_all[i]['fluents']
    
    for fluent in fluents:
        start = fluents[fluent]['start_time']
        end = fluents[fluent]['end_time']
        fluent_no = fluents[fluent]['fluent_no']
        for index, row in mp_participant.iterrows():
            time = row['Frame']
            hand_position_X = row['Wrist_X']
            hand_position_Y = row['Wrist_Y'] 
            if start <= time and time <= end:           
                disfluency = 0
                mp_participant_fluent_handX = hand_position_X
                mp_participant_fluent_handY = hand_position_Y
                all_records.append([participant_id, disfluency, fluent_no, mp_participant_partial_handX, mp_participant_partial_handY]) 

                
all_records_dataframe = pd.DataFrame(all_records, columns = ['ParticipantID', 'Disfluency', 'EventNo', 'Wrist_X', 'Wrist_Y'])
                
# for interval in intervalList:
     # for i in range(interval.start_time, interval.end_time):

In [8]:
inter = sessions_mp_dataframe[(sessions_mp_dataframe['Participant'] == '1_A')]
inter

,Frame,Wrist_X,Wrist_Y,Participant,Session
0,0.00,0.260420,0.418896,1_A,1
1,0.04,0.252418,0.576465,1_A,1
2,0.08,0.260182,0.420576,1_A,1
3,0.08,0.252568,0.576337,1_A,1
4,0.12,0.253143,0.576375,1_A,1
...,...,...,...,...,...
50108,1631.76,0.252350,0.587557,1_A,1
50109,1631.80,0.267880,0.453328,1_A,1
50110,1631.80,0.252214,0.587440,1_A,1
50111,1631.84,0.252288,0.587518,1_A,1


In [11]:
aa_turns = tg.get_tier_by_name('A-turns')
aa_turns_intervals = aa_turns.intervals
aa_turns_intervals_df = pd.DataFrame(aa_turns_intervals, columns = ['Interval'])
aa_turns_intervals_df.to_csv('r2_a_turns.csv')

In [12]:
bb_turns = tg.get_tier_by_name('B-turns')
bb_turns_intervals = bb_turns.intervals
bb_turns_intervals_df = pd.DataFrame(bb_turns_intervals, columns = ['Interval'])
bb_turns_intervals_df.to_csv('r8_b_turns.csv')

In [51]:
aa = tg.get_tier_by_name('A-utts')

In [8]:
aa.delete_annotations_with_text(pattern='laughter', n=0, regex=True)
# aa.delete_annotations_between_timepoints(1523.82372647, 2345.6106784, left_overlap=False, right_overlap=False)

In [66]:
aa_partials = aa.get_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
#aa_partials[0].start_time
#aa_partials[-1]
# 'person' : a 
# 'condition': 1

# aa_partials[i].start_time 
    #end_time = aa_partials[i].end_time
    #print(start_time)

for i, interval in enumerate(aa_partials):
    start_time = interval.start_time
    end_time = interval.end_time 
    partial_no = i
    disfluency = 1 
    print(end_time)

781.390968945
816.882309879
960.796162393
993.29082074
1041.9495078
1051.18035175
1060.06286609
1177.12675752
1202.74713125
1259.65603323
1420.32007705
1470.12056859
1525.37407423
1565.38520732
1574.57769709
1576.46599566


In [3]:
# section as in aa_partials or aa_flu or part?
def iterate_annotations(participant, section, label):
    for i,interval in enumerate(section):
        start_time = interval.start_time
        end_time = interval.end_time 
        annotation_type = i
        label = label 
        print(end_time)
        
    yield{'start_time': start_time, 'end_time': end_time}

In [10]:
bb = tg.get_tier_by_name('B-utts')
bb.delete_annotations_with_text(pattern='laughter', n=0, regex=True)
#bb.delete_annotations_between_timepoints(1321.59253469, 2345.6106784, left_overlap=False, right_overlap=False)

In [11]:
comm = tg.get_tier_by_name('Comments')
comm_intervals = comm.intervals
comm_intervals

[Interval(1437.91281444, 1439.25272848, "Should breathing tags always be included in utterances or can/should they be separated?"),
 Interval(1526.28133631, 1527.82048776, "breathing or laughter?"),
 Interval(1582.01668895, 1586.02617645, "Not entirely sure about the laughter annotation"),
 Interval(1613.27225495, 1613.42015013, "mark lip opening as <nonverbal/>?"),
 Interval(1810.56079535, 1810.68326365, "filled pause?"),
 Interval(1821.8956005, 1822.44125885, "laughter offset is far away from original laughter - should utterance be extend to include offset?"),
 Interval(1881.36830454, 1882.51958038, "correctly marked as first repair and then abandonment?"),
 Interval(2089.23721446, 2090.66715159, "Is this a DF?"),
 Interval(2286.84069142, 2288.24299733, "Should "held closures" be marked?"),
 Interval(2291.83573985, 2297.13333995, "deletion marked correctly?")]

In [12]:
bb_partials = bb.get_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
bb_partials

[Interval(1543.78406429, 1550.10325104, "<nv type='breathing'/> <v='einer'> 'ner </v> Und mit den Gerüchen . ist das auch so <v='eine'> 'ne </v> Sache ne / wenn man kocht dann (will ja <v='nicht'> nich' </v> + . soll ja <v='nicht'> nich' </v>) die ganze Wohnung so . stinken"),
 Interval(1556.96928762, 1558.85645498, "Ja cool (so <v='eine'> 'ne </v> + ja so <v='eine'> 'ne </v>) Kücheninsel"),
 Interval(1592.13912282, 1594.25123789, "(Aber die +) ja die muss dann auch Kakao können genau"),
 Interval(1620.61438304, 1624.08710421, "(Und die man auch so + {also} die so) richtig . ömmelmäßig ist so"),
 Interval(1625.73506264, 1628.84810069, "Ja so ganz breit so (dass + dass) sie bis hier geht oder so"),
 Interval(1675.22938955, 1679.04677786, "Oder man macht auch (so <v='einen'> 'n </v> + . so <v='einen'> 'n </v>) Bar-Bereich mit so Durchreiche-mäßig"),
 Interval(1787.30729543, 1790.47546804, "Na grau . ja aber (so . + (so + sowas)) Freundliches"),
 Interval(1795.58429413, 1797.14241181, "Mi

In [13]:
bb.delete_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
bb.delete_annotations_with_text(pattern='<p=', n=0, regex=True)
bb.delete_annotations_with_text(pattern='breathing', n=0, regex=True)

bb_intervals = bb.intervals
bb_intervals

[Interval(1440.98743133, 1441.8876771, "Auf jeden Fall -"),
 Interval(1442.62345489, 1444.66632029, "Ja Wohnzimmer . Küche Bad <v='ist'> is' </v> klar"),
 Interval(1447.15930858, 1447.1797904, "{Also} dass jede von uns eins hat"),
 Interval(1457.98289541, 1460.25947846, "Oder so <v='ein'> 'n </v> Kinosaa:l oder"),
 Interval(1468.41529022, 1472.81430715, "Oh Sauna ist auch cool / aber das könnten wir wenn wir das Bad groß genug machen auch im Bad unterbringen"),
 Interval(1473.29905487, 1474.38108103, "So <v='ein'> 'n </v> Sauna-Bereich"),
 Interval(1480.62387054, 1482.10408234, "Ja oder wirklich Gäste-Zimmer"),
 Interval(1482.48495555, 1484.19022878, "Wenn mal unsere Eltern kommen oder"),
 Interval(1485.71372162, 1486.13787588, "Mhm"),
 Interval(1491.56698171, 1491.78338694, "Ja"),
 Interval(1492.00844839, 1492.94331899, "<v='das ist'> Da's </v> auch voll schön"),
 Interval(1493.43672292, 1494.82171641, "Das können <v='wir'> wa </v> schon mal festhalten"),
 Interval(1498.36376862, 1498

In [51]:
# aa_partial2 = aa.get_annotations_with_text(pattern='\+', n=0, regex=True)

In [7]:
# YOU NEED TO FIND A WAY TO SEARCH + WITH REGEX GOOGLE

aa_just_partials = aa.get_annotations_with_text(pattern='<p=', n=0, regex=True)
aa_just_partials

[]

In [14]:
aa.delete_annotations_with_text(pattern='(?<=\+).*', n=0, regex=True)
aa.delete_annotations_with_text(pattern='<p=', n=0, regex=True)
bb.delete_annotations_with_text(pattern='breathing', n=0, regex=True)
aa_intervals = aa.intervals

In [15]:
aa_intervals

[Interval(1445.23763011, 1445.45403534, "Ja"),
 Interval(1448.94415029, 1449.20383657, "Ja"),
 Interval(1453.93878306, 1454.58799876, "Brauchen wir sowas"),
 Interval(1454.76874879, 1455.20901321, "<nv type='breathing'/>"),
 Interval(1463.52453196, 1465.09996205, "Ja das stimmt wir haben voll viel Geld eigentlich"),
 Interval(1472.54596466, 1473.16921173, "Ja das stimmt"),
 Interval(1474.89179738, 1475.23804576, "hm"),
 Interval(1475.93881866, 1477.40722786, "<nv type='breathing'/> Ja <v='willst du'> willste </v> so <v='ein'> 'n </v> Extra-Zimmer"),
 Interval(1477.46034643, 1479.25452688, "Obwohl was machen wir dann da sind wir dann da"),
 Interval(1484.25082225, 1485.51462881, "Ja das <v='ist'> is' </v> vielleicht gar <v='nicht'> nich' </v> so schlecht"),
 Interval(1499.26401438, 1499.80935557, "Juhu"),
 Interval(1504.94798653, 1506.91776601, "Tja .. ich musste genug leiden"),
 Interval(1511.65574173, 1512.10449798, "Oh man"),
 Interval(1512.9528065, 1515.10820262, "Oh ich hoffe ich m

In [16]:
dfa_flu_intervals = pd.DataFrame(aa_intervals, columns = ['Interval'])
dfa_flu_intervals.to_csv('r3_a_filtered_flu_intervals.csv')

In [18]:
dfb_flu_intervals = pd.DataFrame(bb_intervals, columns = ['Interval'])
dfb_flu_intervals.to_csv('r3_b_filtered_flu_intervals.csv')

In [19]:
# dfb_intervals = pd.DataFrame(bb_intervals, columns = ['Interval'])
# dfb_intervals.to_csv('r3_b_all_intervals.csv')

dfb_partials = pd.DataFrame(bb_partials, columns = ['Partial'])
dfb_partials.to_csv('r3_b_partials_filt.csv')

In [20]:
# dfa_intervals = pd.DataFrame(aa_intervals, columns = ['Interval'])
# dfa_intervals.to_csv('r3_a_all_intervals.csv')

dfa_partials = pd.DataFrame(aa_partials, columns = ['Partial'])
dfa_partials.to_csv('r3_a_partials_filt.csv')

In [ ]:
# parse + {pluses}
# get start time of utterances containing + (this will not give me the exact disfluency time)

In [18]:
# \w is words or strings \s matches any whitespace character such as space
# <p s“Wohnzimmer”>Wohn-</p> + . { ja also } ( die + ( die + das ) ) {F a ̈h} ... Wohnzimmer  like this


# adapted from https://github.com/Nagasaki45/dd-tag-parser/blob/master/dd_tag_parser.py

_PATTERN = re.compile(
    r'<'
    r'(?P<tag>\w*)'
    r'(?P<attributes>[\w\s="].*)?'
    r'/>'
)

_ATTRIBUTES_PATTERN = re.compile(
    r'(?P<key>\w+)'
    r'='
    r'"(?P<value>\w+)"'
)


def _parse_full_string(full_string):
    for single_string in _PATTERN.findall(full_string):
        yield _parse_single_tag_string(single_string)


def _parse_single_tag_string(single_string):
    tag, raw_attributes = single_string
    return {'tag': tag, 'attributes': _parse_attributes(raw_attributes)}


def _parse_attributes(raw_attributes):
    return dict(_ATTRIBUTES_PATTERN.findall(raw_attributes))


def parse(tags):
    return list(_parse_full_string(tags))

In [22]:
# this dictionary framework

for result in dd:
    result['disf_tag'] = parse(result['disf_tag'])

TypeError: 'Interval' object is not subscriptable